# Proposition Chunking for Enhanced RAG

## Setting Up the Environment

In [1]:
import numpy as np
import json
import fitz
import re

In [17]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [18]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로

    Returns:
        str: PDF에서 추출된 전체 텍스트
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 전체 텍스트를 저장할 문자열 초기화

    # 각 페이지를 순회하며 텍스트 추출
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]               # 해당 페이지 가져오기
        text = page.get_text("text")         # 텍스트 형식으로 내용 추출
        all_text += text                     # 추출된 텍스트 누적

    # 추출된 전체 텍스트 반환
    return all_text

## Chunking the Extracted Text

In [19]:
def chunk_text(text, chunk_size=800, overlap=100):
    """
    텍스트를 일정 길이로 겹치게 분할합니다.

    Args:
        text (str): 분할할 원본 텍스트
        chunk_size (int): 각 청크의 문자 수 (기본: 800자)
        overlap (int): 청크 간 중첩 길이 (기본: 100자)

    Returns:
        List[Dict]: 텍스트와 메타데이터를 포함한 청크 딕셔너리 리스트
    """
    chunks = []  # 청크들을 저장할 빈 리스트 초기화

    # 지정된 청크 크기와 중첩 길이에 따라 텍스트 분할
    for i in range(0, len(text), chunk_size - overlap):
        chunk = text[i:i + chunk_size]  # 해당 범위만큼 청크 추출
        if chunk:  # 빈 청크는 제외
            chunks.append({
                "text": chunk,  # 청크 본문
                "chunk_id": len(chunks) + 1,  # 청크 고유 ID
                "start_char": i,  # 청크 시작 인덱스
                "end_char": i + len(chunk)  # 청크 끝 인덱스
            })

    # 생성된 청크 수 출력
    print(f"Total {len(chunks)}개의 텍스트 청크가 생성되었습니다.")
    return chunks  # 청크 리스트 반환

## Simple Vector Store Implementation

In [20]:
import numpy as np

class SimpleVectorStore:
    """
    NumPy를 사용한 간단한 벡터 저장소 구현입니다.
    """
    def __init__(self):
        """
        벡터 저장소를 초기화합니다.
        """
        self.vectors = []  # 임베딩 벡터를 저장할 리스트
        self.texts = []  # 원본 텍스트를 저장할 리스트
        self.metadata = []  # 각 텍스트의 메타데이터를 저장할 리스트
    
    def add_item(self, text, embedding, metadata=None):
        """
        항목을 벡터 저장소에 추가합니다.

        Args:
        text (str): 원본 텍스트
        embedding (List[float]): 임베딩 벡터
        metadata (dict, 선택): 추가적인 메타데이터
        """
        self.vectors.append(np.array(embedding))  # 임베딩을 넘파이 배열로 변환하여 vectors 리스트에 추가
        self.texts.append(text)  # 원본 텍스트를 texts 리스트에 추가
        self.metadata.append(metadata or {})  # 메타데이터를 metadata 리스트에 추가, 없으면 빈 딕셔너리 사용
    
    def similarity_search(self, query_embedding, k=5, filter_func=None):
        """
        쿼리 임베딩과 가장 유사한 항목들을 찾습니다.

        Args:
        query_embedding (List[float]): 쿼리 임베딩 벡터
        k (int): 반환할 결과 수
        filter_func (callable, 선택): 결과를 필터링할 함수

        Returns:
        List[Dict]: 텍스트와 메타데이터, 유사도 점수를 포함한 상위 k개 유사 항목 리스트
        """
        if not self.vectors:
            return []  # 저장된 벡터가 없다면 빈 리스트 반환
        
        # 쿼리 임베딩을 넘파이 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 사용하여 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            # 필터 함수가 있다면 해당 메타데이터를 기준으로 필터링
            if filter_func and not filter_func(self.metadata[i]):
                continue
                
            # 코사인 유사도 계산
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # 인덱스와 유사도 점수를 추가
        
        # 유사도를 기준으로 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개의 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # 텍스트 추가
                "metadata": self.metadata[idx],  # 메타데이터 추가
                "similarity": score  # 유사도 점수 추가
            })
        
        return results  # 상위 k개 결과 리스트 반환

## Creating Embeddings

In [21]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    주어진 텍스트에 대해 임베딩을 생성합니다.

    Args:
        text (str 또는 List[str]): 임베딩을 생성할 입력 텍스트(또는 텍스트 리스트)
        model (str): 사용할 임베딩 모델 이름

    Returns:
        List[float] 또는 List[List[float]]: 생성된 임베딩 벡터 또는 벡터 리스트
    """
    # 입력이 문자열 하나일 수도 있고, 문자열 리스트일 수도 있으므로 리스트 형태로 통일
    input_text = text if isinstance(text, list) else [text]

    # 지정된 모델을 사용하여 임베딩 생성 요청
    response = client.embeddings.create(
        model=model,
        input=input_text
    )

    # 입력이 단일 문자열이었을 경우, 첫 번째 임베딩만 반환
    if isinstance(text, str):
        return response.data[0].embedding

    # 여러 문자열일 경우, 모든 임베딩 리스트 반환
    return [item.embedding for item in response.data]

## Proposition Generation

In [22]:
def generate_propositions(chunk):
    """
    텍스트 청크에서 원자적이고 자족적인 명제들을 생성합니다.

    매개변수:
        chunk (Dict): 텍스트와 메타데이터를 포함한 청크

    반환값:
        List[str]: 생성된 명제 리스트
    """
    # 명제 생성을 위한 시스템 프롬프트 정의
    system_prompt = """다음 텍스트를 단순하고 자족적인 명제들로 분해해 주세요.
    각 명제는 다음 기준을 충족해야 합니다:

    1. 하나의 사실만 표현할 것: 각 명제는 하나의 구체적인 사실이나 주장만을 담아야 합니다.
    2. 문맥 없이 이해 가능할 것: 명제는 자족적이어야 하며, 추가적인 문맥 없이도 이해되어야 합니다.
    3. 대명사 대신 전체 이름 사용할 것: 대명사나 모호한 지시어 대신, 전체 엔터티 이름을 사용하세요.
    4. 관련 날짜/수식어 포함: 필요한 경우 명확성을 위해 날짜, 시간, 수식어를 포함하세요.
    5. 하나의 주어-술어 관계만 포함: 연결사 없이 하나의 주어와 그에 해당하는 동작 또는 속성만 표현하세요.

    명제 리스트만 출력하고, 그 외의 설명이나 추가 텍스트는 포함하지 마세요."""


    # 사용자 프롬프트: 명제로 변환할 텍스트 청크
    user_prompt = f"명제(proposition)로 변환할 텍스트:\n\n{chunk['text']}"
    
    # 모델 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0  # 창의성보다 정확성을 중시
    )
    
    # 응답으로부터 명제 줄 단위 추출
    raw_propositions = response.choices[0].message.content.strip().split('\n')
    
    # 불필요한 번호, 기호 등 제거하여 명제 정리
    clean_propositions = []
    for prop in raw_propositions:
        cleaned = re.sub(r'^\s*(\d+\.|\-|\*)\s*', '', prop).strip()
        if cleaned and len(cleaned) > 10:  # 너무 짧거나 빈 명제는 제외
            clean_propositions.append(cleaned)
    
    return clean_propositions

## Quality Checking for Propositions

In [23]:
def evaluate_proposition(proposition, original_text):
    """
    Evaluate a proposition's quality based on accuracy, clarity, completeness, and conciseness.
    
    Args:
        proposition (str): The proposition to evaluate
        original_text (str): The original text for comparison
        
    Returns:
        Dict: Scores for each evaluation dimension
    """
    # System prompt to instruct the AI on how to evaluate the proposition
    system_prompt = """You are an expert at evaluating the quality of propositions extracted from text.
    Rate the given proposition on the following criteria (scale 1-10):

    - Accuracy: How well the proposition reflects information in the original text
    - Clarity: How easy it is to understand the proposition without additional context
    - Completeness: Whether the proposition includes necessary details (dates, qualifiers, etc.)
    - Conciseness: Whether the proposition is concise without losing important information

    The response must be in valid JSON format with numerical scores for each criterion:
    {"accuracy": X, "clarity": X, "completeness": X, "conciseness": X}
    """

    # User prompt containing the proposition and the original text
    user_prompt = f"""Proposition: {proposition}

    Original Text: {original_text}

    Please provide your evaluation scores in JSON format."""

    # Generate response from the model
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"},
        temperature=0
    )
    
    # Parse the JSON response
    try:
        scores = json.loads(response.choices[0].message.content.strip())
        return scores
    except json.JSONDecodeError:
        # Fallback if JSON parsing fails
        return {
            "accuracy": 5,
            "clarity": 5,
            "completeness": 5,
            "conciseness": 5
        }
    
    
import json

def evaluate_proposition(proposition, original_text):
    """
    명제를 정확성, 명확성, 완전성, 간결성 기준으로 평가합니다.

    Args:
        proposition (str): 평가할 명제
        original_text (str): 명제가 추출된 원본 텍스트

    Returns:
        Dict: 각 평가 기준에 대한 점수 (1~10)
    """
    # 평가 기준을 설명하는 시스템 프롬프트
    system_prompt = """귀하는 텍스트에서 추출한 명제의 품질을 평가하는 전문가입니다.
    다음 기준에 따라 주어진 명제를 평가하세요(1~10점 척도):

    - Accuracy: 명제가 원문 텍스트의 정보를 얼마나 잘 반영하는지 여부
    - Clarity: 추가적인 맥락 없이도 명제를 얼마나 쉽게 이해할 수 있는지 여부
    - Completeness: 명제에 필요한 세부 사항(날짜, 한정어 등)이 포함되어 있는지 여부
    - Conciseness: 명제가 중요한 정보를 놓치지 않고 간결한지 여부

    응답은 각 기준에 대한 수치 점수가 포함된 유효한 JSON 형식이어야 합니다:
        {"accuracy": X, "clarity": X, "completeness": X, "conciseness": X}
        """

    # 사용자 입력: 명제와 원문
    user_prompt = f"""Proposition: {proposition}

    Original Text: {original_text}

    평가 점수를 JSON 형식으로 제공해 주세요."""

    # LLM을 호출하여 평가 점수 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"},  # JSON 형식 응답 요청
        temperature=0  # 일관된 평가를 위해 창의성 최소화
    )
    
    # 모델 응답에서 JSON 파싱 시도
    try:
        scores = json.loads(response.choices[0].message.content.strip())
        return scores
    except json.JSONDecodeError:
        # JSON 파싱 실패 시, 기본 점수 반환
        return {
            "accuracy": 5,
            "clarity": 5,
            "completeness": 5,
            "conciseness": 5
        }

## Complete Proposition Processing Pipeline

In [24]:
def process_document_into_propositions(pdf_path, chunk_size=800, chunk_overlap=100, 
                                      quality_thresholds=None):
    """
    문서를 처리하여 품질 기준을 통과한 명제들을 생성합니다.

    Args:
        pdf_path (str): PDF 파일 경로
        chunk_size (int): 각 청크의 문자 수
        chunk_overlap (int): 청크 간 중첩 문자 수
        quality_thresholds (Dict): 명제 품질 평가 기준 점수

    Returns:
        Tuple[List[Dict], List[Dict]]: 원본 청크 리스트, 품질 필터링된 명제 리스트
    """
    # 품질 기준이 없을 경우 기본 기준 설정
    if quality_thresholds is None:
        quality_thresholds = {
            "accuracy": 7,
            "clarity": 7,
            "completeness": 7,
            "conciseness": 7
        }
    
    # PDF에서 텍스트 추출
    text = extract_text_from_pdf(pdf_path)
    
    # 추출된 텍스트를 청크 단위로 분할
    chunks = chunk_text(text, chunk_size, chunk_overlap)
    
    all_propositions = []  # 전체 명제 저장 리스트 초기화

    print("청크로부터 명제를 생성 중...")
    for i, chunk in enumerate(chunks):
        print(f"{i+1}/{len(chunks)} 번째 청크 처리 중...")

        # 현재 청크에 대해 명제 생성
        chunk_propositions = generate_propositions(chunk)
        print(f"생성된 명제 수: {len(chunk_propositions)}")

        # 각 명제를 메타데이터와 함께 저장
        for prop in chunk_propositions:
            proposition_data = {
                "text": prop,
                "source_chunk_id": chunk["chunk_id"],
                "source_text": chunk["text"]
            }
            all_propositions.append(proposition_data)

    # 명제 품질 평가 단계
    print("\n명제 품질 평가 중...")
    quality_propositions = []  # 품질 기준을 통과한 명제 리스트

    for i, prop in enumerate(all_propositions):
        if i % 10 == 0:  # 10개마다 진행 상황 출력
            print(f"{i+1}/{len(all_propositions)} 번째 명제 평가 중...")

        # 해당 명제의 품질 점수 평가
        scores = evaluate_proposition(prop["text"], prop["source_text"])
        prop["quality_scores"] = scores

        # 모든 기준 점수를 통과하는지 확인
        passes_quality = True
        for metric, threshold in quality_thresholds.items():
            if scores.get(metric, 0) < threshold:
                passes_quality = False
                break

        if passes_quality:
            quality_propositions.append(prop)
        else:
            print(f"품질 기준 미달 명제: {prop['text'][:50]}...")

    print(f"\n최종 통과 명제 수: {len(quality_propositions)}/{len(all_propositions)}")

    # 결과 반환: 전체 청크와, 품질 기준 통과 명제
    return chunks, quality_propositions

## Building Vector Stores for Both Approaches

In [25]:
def build_vector_stores(chunks, propositions):
    """
    문서 청크와 명제 기반의 벡터 저장소를 생성합니다.

    Args:
        chunks (List[Dict]): 원본 문서 청크 리스트
        propositions (List[Dict]): 품질 필터링된 명제 리스트

    Returns:
        Tuple[SimpleVectorStore, SimpleVectorStore]: 청크 기반, 명제 기반 벡터 저장소
    """
    # 청크 기반 벡터 저장소 생성
    chunk_store = SimpleVectorStore()
    
    # 청크 텍스트 추출 및 임베딩 생성
    chunk_texts = [chunk["text"] for chunk in chunks]
    print(f"{len(chunk_texts)}개의 청크에 대해 임베딩 생성 중...")
    chunk_embeddings = create_embeddings(chunk_texts)
    
    # 메타데이터 생성 후 벡터 저장소에 추가
    chunk_metadata = [{"chunk_id": chunk["chunk_id"], "type": "chunk"} for chunk in chunks]
    chunk_store.add_items(chunk_texts, chunk_embeddings, chunk_metadata)
    
    # 명제 기반 벡터 저장소 생성
    prop_store = SimpleVectorStore()
    
    # 명제 텍스트 추출 및 임베딩 생성
    prop_texts = [prop["text"] for prop in propositions]
    print(f"{len(prop_texts)}개의 명제에 대해 임베딩 생성 중...")
    prop_embeddings = create_embeddings(prop_texts)
    
    # 명제 메타데이터 생성 후 저장소에 추가
    prop_metadata = [
        {
            "type": "proposition", 
            "source_chunk_id": prop["source_chunk_id"],
            "quality_scores": prop["quality_scores"]
        } 
        for prop in propositions
    ]
    prop_store.add_items(prop_texts, prop_embeddings, prop_metadata)
    
    # 두 개의 저장소 반환
    return chunk_store, prop_store

## Query and Retrieval Functions

In [26]:
def retrieve_from_store(query, vector_store, k=5):
    """
    쿼리를 기반으로 벡터 저장소에서 관련 항목들을 검색합니다.

    Args:
        query (str): 사용자 쿼리
        vector_store (SimpleVectorStore): 검색 대상 벡터 저장소
        k (int): 반환할 결과 수 (기본값: 5개)

    Returns:
        List[Dict]: 유사도 점수와 메타데이터가 포함된 검색 결과 리스트
    """
    # 쿼리를 임베딩으로 변환
    query_embedding = create_embeddings(query)

    # 벡터 저장소에서 상위 k개 유사 항목 검색
    results = vector_store.similarity_search(query_embedding, k=k)

    return results

In [27]:
def compare_retrieval_approaches(query, chunk_store, prop_store, k=5):
    """
    하나의 쿼리에 대해 청크 기반과 명제 기반 검색 방식을 비교합니다.

    Args:
        query (str): 사용자 검색 쿼리
        chunk_store (SimpleVectorStore): 청크 기반 벡터 저장소
        prop_store (SimpleVectorStore): 명제 기반 벡터 저장소
        k (int): 각 저장소에서 검색할 결과 수

    Returns:
        Dict: 두 검색 방식의 결과를 포함한 비교 정보
    """
    print(f"\n=== 쿼리: {query} ===")
    
    # 명제 기반 검색
    print("\n[명제 기반 검색 수행 중...]")
    prop_results = retrieve_from_store(query, prop_store, k)
    
    # 청크 기반 검색
    print("[청크 기반 검색 수행 중...]")
    chunk_results = retrieve_from_store(query, chunk_store, k)
    
    # 명제 기반 결과 출력
    print("\n=== 명제 기반 결과 ===")
    for i, result in enumerate(prop_results):
        print(f"{i+1}) {result['text']} (유사도: {result['similarity']:.4f})")
    
    # 청크 기반 결과 출력
    print("\n=== 청크 기반 결과 ===")
    for i, result in enumerate(chunk_results):
        # 너무 긴 텍스트는 150자까지만 출력
        truncated_text = result['text'][:150] + "..." if len(result['text']) > 150 else result['text']
        print(f"{i+1}) {truncated_text} (유사도: {result['similarity']:.4f})")
    
    # 결과 딕셔너리로 반환
    return {
        "query": query,
        "proposition_results": prop_results,
        "chunk_results": chunk_results
    }

## Response Generation and Evaluation

In [28]:
def generate_response(query, results, result_type="proposition"):
    """
    검색된 결과를 기반으로 AI 응답을 생성합니다.

    Args:
        query (str): 사용자 질문
        results (List[Dict]): 검색된 항목 리스트
        result_type (str): 검색 결과의 유형 ('proposition' 또는 'chunk')

    Returns:
        str: 생성된 AI 응답
    """
    # 검색된 텍스트들을 하나의 문맥(context) 문자열로 결합
    context = "\n\n".join([result["text"] for result in results])
    
    # AI에게 응답 지침을 주는 시스템 프롬프트 정의
    system_prompt = f"""당신은 검색된 정보를 바탕으로 질문에 답하는 AI 어시스턴트입니다.
    당신의 답변은 지식 기반에서 검색된 다음의 {result_type}들을 기반으로 해야 합니다.
    검색된 정보만으로 질문에 답할 수 없다면, 그 한계를 명확히 인정해야 합니다."""

    # 사용자 프롬프트: 질문 + 검색된 문맥
    user_prompt = f"""Query: {query}

    Retrieved {result_type}s:
    {context}

    검색된 정보를 바탕으로 쿼리에 답변해 주세요."""

    # OpenAI 또는 호환 클라이언트를 통해 응답 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2  # 비교적 낮은 창의성 (정보 충실도 중시)
    )
    
    # 응답 텍스트만 반환
    return response.choices[0].message.content


In [29]:
def evaluate_responses(query, prop_response, chunk_response, reference_answer=None):
    """
    명제 기반 응답과 청크 기반 응답을 비교 평가합니다.

    Args:
        query (str): 사용자 질문
        prop_response (str): 명제 기반 검색으로 생성된 응답
        chunk_response (str): 청크 기반 검색으로 생성된 응답
        reference_answer (str, 선택): 비교용 정답 (있을 경우 정확성 기준 제공)

    Returns:
        str: 평가 분석 결과 (자연어 텍스트)
    """
    # 평가 시스템 프롬프트 정의: 평가 기준과 방식 설명
    system_prompt = """당신은 정보 검색 시스템 평가 전문가입니다.
    하나의 쿼리에 대해 생성된 두 개의 응답을 비교하세요. 
    하나는 명제 기반 검색(proposition-based retrieval), 다른 하나는 청크 기반 검색(chunk-based retrieval)에 의해 생성된 응답입니다.

    다음 기준에 따라 두 응답을 평가하십시오:
    1. 정확성(Accuracy): 어느 응답이 사실적으로 더 정확한 정보를 제공하는가?
    2. 관련성(Relevance): 어느 응답이 쿼리의 의도에 더 잘 부합하는가?
    3. 간결성(Conciseness): 어느 응답이 핵심을 놓치지 않으면서 더 간결하게 설명하는가?
    4. 명확성(Clarity): 어느 응답이 더 이해하기 쉬운가?

    각 방식의 강점과 약점을 구체적으로 서술하십시오."""


    # 사용자 프롬프트 구성: 쿼리 및 두 응답 포함
    user_prompt = f"""Query: {query}

    Response from Proposition-Based Retrieval:
    {prop_response}

    Response from Chunk-Based Retrieval:
    {chunk_response}"""

    # 참조 정답이 제공된 경우, 프롬프트에 포함하여 사실성 비교 가능하도록 함
    if reference_answer:
        user_prompt += f"""

    Reference Answer (for factual checking):
    {reference_answer}"""

    # 사용자에게 비교 평가 요청
    user_prompt += """
    이 두 가지 응답을 자세히 비교하여 어떤 접근 방식이 더 나은 성과를 냈는지, 그 이유는 무엇인지 설명해 주세요."""

    # 평가 분석 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0  # 일관된 평가를 위해 창의성 최소화
    )
    
    # 평가 결과 텍스트 반환
    return response.choices[0].message.content

## Complete End-to-End Evaluation Pipeline

In [31]:
def run_proposition_chunking_evaluation(pdf_path, test_queries, reference_answers=None):
    """
    명제 기반 청크화 vs 일반 청크화에 대한 종합 평가를 실행합니다.

    Args:
        pdf_path (str): PDF 파일 경로
        test_queries (List[str]): 테스트할 질문 리스트
        reference_answers (List[str], 선택): 정답 리스트 (있을 경우 정확성 평가에 사용)

    Returns:
        Dict: 평가 결과, 전체 분석, 명제/청크 개수 포함
    """
    print("***명제 기반 청크화 평가 시작***\n")
    
    # 문서 처리 → 청크 및 명제 생성
    chunks, propositions = process_document_into_propositions(pdf_path)
    
    # 벡터 저장소 구축 (청크용, 명제용)
    chunk_store, prop_store = build_vector_stores(chunks, propositions)
    
    results = []  # 전체 평가 결과 저장 리스트
    
    # 쿼리별 테스트 실행
    for i, query in enumerate(test_queries):
        print(f"\n\n***쿼리 {i+1}/{len(test_queries)} 테스트 중***")
        print(f"질문: {query}")
        
        # 청크 기반 vs 명제 기반 검색 결과 비교
        retrieval_results = compare_retrieval_approaches(query, chunk_store, prop_store)
        
        # 명제 기반 결과로 응답 생성
        print("\n명제 기반 응답 생성 중...")
        prop_response = generate_response(
            query, 
            retrieval_results["proposition_results"], 
            "proposition"
        )
        
        # 청크 기반 결과로 응답 생성
        print("청크 기반 응답 생성 중...")
        chunk_response = generate_response(
            query, 
            retrieval_results["chunk_results"], 
            "chunk"
        )
        
        # 정답이 있다면 포함
        reference = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]
        
        # 두 응답 평가
        print("\n응답 비교 평가 중...")
        evaluation = evaluate_responses(query, prop_response, chunk_response, reference)
        
        # 현재 쿼리 결과 정리
        query_result = {
            "query": query,
            "proposition_results": retrieval_results["proposition_results"],
            "chunk_results": retrieval_results["chunk_results"],
            "proposition_response": prop_response,
            "chunk_response": chunk_response,
            "reference_answer": reference,
            "evaluation": evaluation
        }
        results.append(query_result)
        
        # 결과 출력
        print("\n***명제 기반 응답***")
        print(prop_response)
        print("\n***청크 기반 응답***")
        print(chunk_response)
        print("\n***평가 결과***")
        print(evaluation)
    
    # 전체 종합 분석 생성
    print("\n\n***전체 분석 생성 중***")
    overall_analysis = generate_overall_analysis(results)
    print("\n" + overall_analysis)
    
    # 최종 결과 반환
    return {
        "results": results,
        "overall_analysis": overall_analysis,
        "proposition_count": len(propositions),
        "chunk_count": len(chunks)
    }

In [32]:
def generate_overall_analysis(results):
    """
    명제 기반 vs 청크 기반 접근 방식의 종합 분석을 생성합니다.

    Args:
        results (List[Dict]): 각 테스트 쿼리의 평가 결과 리스트

    Returns:
        str: 종합 분석 결과 (자연어 텍스트)
    """
    # 시스템 프롬프트: LLM에게 평가자 역할 및 비교 관점을 지시
    system_prompt = """당신은 정보 검색 시스템을 평가하는 전문가입니다.
    여러 테스트 쿼리를 바탕으로, RAG(Retrieval-Augmented Generation) 시스템에서 
    명제 기반 검색(proposition-based retrieval)과 청크 기반 검색(chunk-based retrieval)을 비교하여 종합 분석을 제공하세요.

    다음 사항에 중점을 두어 평가하십시오:
    1. 명제 기반 검색이 더 우수한 경우
    2. 청크 기반 검색이 더 우수한 경우
    3. 각 접근 방식의 전반적인 강점과 약점
    4. 각 접근 방식을 어떤 상황에서 사용하는 것이 좋은지에 대한 추천"""


    # 각 쿼리 평가의 요약 내용을 생성
    evaluations_summary = ""
    for i, result in enumerate(results):
        evaluations_summary += f"Query {i+1}: {result['query']}\n"
        evaluations_summary += f"Evaluation Summary: {result['evaluation'][:200]}...\n\n"  # 앞부분만 요약 출력

    # 사용자 프롬프트: 전체 평가 요약을 기반으로 종합 분석 요청
    user_prompt = f"""다음은 명제 기반 검색(proposition-based retrieval)과 청크 기반 검색(chunk-based retrieval)에 대한 {len(results)}개의 쿼리 평가 결과입니다. 
    이 평가들을 바탕으로 두 접근 방식에 대한 종합적인 비교 분석을 작성해 주세요:

    {evaluations_summary}

    명제 기반 검색과 청크 기반 검색의 상대적인 강점과 약점을 중심으로,
    RAG(Retrieval-Augmented Generation) 시스템에서 두 방식의 성능을 포괄적으로 분석해 주세요."""


    # LLM을 통해 종합 분석 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0  # 일관성 있는 분석을 위해 창의성 최소화
    )
    
    # 생성된 분석 결과 반환
    return response.choices[0].message.content

## Evaluation of Proposition Chunking

In [33]:
# 처리할 AI 정보 문서의 경로
pdf_path = "dataset/AI_Understanding.pdf"

# AI의 다양한 측면을 평가하기 위한 테스트 쿼리 정의 (현재는 1개 사용)
test_queries = [
    "AI 개발의 주요 윤리적 문제는 무엇인가요?",
]

# 명제 기반 vs 청크 기반 응답의 정확도 비교를 위한 기준 정답 (Reference Answers)
reference_answers = [
    "AI 개발의 주요 윤리적 문제에는 편견과 공정성, 개인정보 보호, 투명성, 책임성, 안전, 오용 또는 유해한 애플리케이션의 가능성 등이 있습니다.",
]

# 평가 실행 (엔드 투 엔드: 문서 처리 → 명제 생성 → 벡터 저장소 → 검색 → 응답 생성 → 평가)
evaluation_results = run_proposition_chunking_evaluation(
    pdf_path=pdf_path,
    test_queries=test_queries,
    reference_answers=reference_answers
)

# 전체 분석 결과 출력
print("\n\n***Overall Analysis***")
print(evaluation_results["overall_analysis"])

***명제 기반 청크화 평가 시작***

Total 24개의 텍스트 청크가 생성되었습니다.
청크로부터 명제를 생성 중...
1/24 번째 청크 처리 중...
생성된 명제 수: 20
2/24 번째 청크 처리 중...
생성된 명제 수: 27
3/24 번째 청크 처리 중...
생성된 명제 수: 17
4/24 번째 청크 처리 중...
생성된 명제 수: 45
5/24 번째 청크 처리 중...
생성된 명제 수: 39
6/24 번째 청크 처리 중...
생성된 명제 수: 24
7/24 번째 청크 처리 중...
생성된 명제 수: 27
8/24 번째 청크 처리 중...
생성된 명제 수: 41
9/24 번째 청크 처리 중...
생성된 명제 수: 23
10/24 번째 청크 처리 중...
생성된 명제 수: 44
11/24 번째 청크 처리 중...
생성된 명제 수: 33
12/24 번째 청크 처리 중...
생성된 명제 수: 34
13/24 번째 청크 처리 중...
생성된 명제 수: 36
14/24 번째 청크 처리 중...
생성된 명제 수: 38
15/24 번째 청크 처리 중...
생성된 명제 수: 36
16/24 번째 청크 처리 중...
생성된 명제 수: 41
17/24 번째 청크 처리 중...
생성된 명제 수: 33
18/24 번째 청크 처리 중...
생성된 명제 수: 30
19/24 번째 청크 처리 중...
생성된 명제 수: 44
20/24 번째 청크 처리 중...
생성된 명제 수: 40
21/24 번째 청크 처리 중...
생성된 명제 수: 35
22/24 번째 청크 처리 중...
생성된 명제 수: 28
23/24 번째 청크 처리 중...
생성된 명제 수: 32
24/24 번째 청크 처리 중...
생성된 명제 수: 21

명제 품질 평가 중...
1/788 번째 명제 평가 중...
품질 기준 미달 명제: 인공 지능은 종종 신화와 소설에 묘사되었다....
11/788 번째 명제 평가 중...
품질 기준 미달 명제: 인공 지능의 편견에 대한 우려가 커지고 있다....
21/788 번째

KeyboardInterrupt: 